In [1]:
import numpy as np
import pandas as pd


In [2]:
credits=pd.read_csv("tmdb_5000_credits.csv")
movie=pd.read_csv("tmdb_5000_movies.csv")

In [3]:
movie = movie.merge(credits,on='title')


In [4]:
movie=movie[['movie_id','genres','title','keywords','overview','cast','crew']]

In [5]:
movie.isnull().sum()

movie_id    0
genres      0
title       0
keywords    0
overview    3
cast        0
crew        0
dtype: int64

In [6]:
movie.dropna(inplace=True)

In [7]:
movie.isnull().sum()

movie_id    0
genres      0
title       0
keywords    0
overview    0
cast        0
crew        0
dtype: int64

In [8]:
movie.duplicated().sum()

0

In [9]:
import ast

In [10]:
def converter1(obj):
    L=[]
    for i in ast.literal_eval(obj) :
        L.append(i['name'])
    return L

In [11]:
movie['genres'] = movie['genres'].apply(converter1)

In [12]:
movie['keywords'] = movie['keywords'].apply(converter1)

In [13]:
def converter2(obj):  
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter<3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L
        

In [14]:
movie['cast']=movie['cast'].apply(converter2)

In [15]:
def converter3(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job'] == "Director":
            L.append(i['name'])
            break
    return L

In [16]:
movie['crew'] = movie['crew'].apply(converter3)

In [17]:
movie.overview=movie.overview.apply(lambda x:x.split(' '))

In [18]:
movie['cast']=movie['cast'].apply(lambda x:[i.replace(" ","")for i in x])
movie['crew']=movie['crew'].apply(lambda x:[i.replace(" ","")for i in x])
movie['genres']=movie['genres'].apply(lambda x:[i.replace(" ","")for i in x])
movie['keywords']=movie['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
    

In [19]:
movie['tags']=movie['overview'] + movie['genres'] + movie['keywords'] + movie['cast'] + movie['crew']

In [20]:
new_df = movie[['movie_id','title','tags']]

In [21]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\Yeshwant\AppData\Local\Temp\ipykernel_944\3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [22]:
import nltk

In [23]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [24]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)     

In [25]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\Yeshwant\AppData\Local\Temp\ipykernel_944\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [26]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\Yeshwant\AppData\Local\Temp\ipykernel_944\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [27]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer(max_features=5000,stop_words='english')
vectors=vectorizer.fit_transform(new_df['tags']).toarray()

In [28]:
vectorizer.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [29]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)

In [30]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distance=enumerate(similarity[movie_index])
    movie_list = sorted(list(distance),reverse=True,key=lambda x:x[1])[1:6]
    for i in movie_list:
        print(new_df.iloc[i[0]].title)
    return 

In [31]:
recommend('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [32]:
import pickle

In [35]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [36]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [37]:
pickle.dump(similarity,open('similarity.pkl','wb'))